# Projet Applied Data Science in Finance - M1 MBFA
## Comparateur de Performance d'ETFs Thématiques

Ce notebook documente le développement d'un projet Python pour l'analyse et la comparaison d'ETFs thématiques. Le projet utilise les bonnes pratiques de programmation, avec une architecture modulaire et une documentation approfondie.

### Objectifs du Projet
- Création d'un ETL pour le chargement et le traitement des données d'ETFs
- Stockage efficace des données dans SQLite
- Interface utilisateur interactive avec Streamlit
- Visualisations avancées avec Plotly
- Documentation complète du code

### Technologies Utilisées
- Python 3.12
- yfinance pour les données financières
- SQLite et SQLAlchemy pour le stockage
- Pandas pour le traitement des données
- Plotly pour les visualisations
- Streamlit pour l'interface utilisateur

## 1. Importation des Bibliothèques
Importons les bibliothèques nécessaires pour notre projet :

In [ ]:
import numpy as np
import pandas as pd
import yfinance as yf
import plotly.express as px
import plotly.graph_objects as go
from sqlalchemy import create_engine
from datetime import datetime, timedelta

# Vérification des versions des bibliothèques
print(f"pandas version: {pd.__version__}")
print(f"numpy version: {np.__version__}")
print(f"yfinance version: {yf.__version__}")

## 2. Chargement des Données depuis YFinance
Nous utilisons la classe `ETFDataLoader` pour charger les données des ETFs depuis yfinance. Cette classe gère :
- Le chargement des métadonnées depuis un CSV
- La récupération des prix historiques
- La mise en cache des données dans SQLite

In [ ]:
# Exemple d'utilisation de la classe ETFDataLoader
from src.etl import ETFDataLoader
from src.repository import ETFRepository

# Créer une instance du repository
repository = ETFRepository('data/etf_analyzer.db')

# Créer une instance du loader
loader = ETFDataLoader(repository)

# Liste d'exemple d'ETFs thématiques
etfs = ['ARKK', 'BOTZ', 'ICLN', 'FINX', 'SKYY']

# Charger les données
data = loader.fetch_multiple_etfs_data(
    tickers=etfs,
    period='2y'
)

# Afficher les premières données pour un ETF
first_etf = list(data.values())[0]
df = pd.DataFrame([{
    'date': price.date,
    'adj_close': price.adj_close,
    'volume': price.volume
} for price in first_etf])

print(f"\nAperçu des données pour {etfs[0]}:")
print(df.head())

## 3. Nettoyage et Transformation des Données
Les données brutes nécessitent un traitement avant analyse :
- Gestion des valeurs manquantes
- Conversion des types de données
- Calcul des rendements et autres métriques

In [ ]:
# Fonction pour transformer les données brutes en DataFrame
def prepare_price_data(etf_data):
    # Créer un DataFrame pour chaque ETF
    dfs = {}
    for ticker, prices in etf_data.items():
        df = pd.DataFrame([
            {
                'date': p.date,
                'adj_close': p.adj_close,
                'volume': p.volume
            } for p in prices
        ])
        df.set_index('date', inplace=True)
        df.sort_index(inplace=True)
        
        # Calculer les rendements journaliers
        df['returns'] = df['adj_close'].pct_change()
        
        # Nettoyer les valeurs manquantes
        df.dropna(inplace=True)
        
        dfs[ticker] = df
    
    return dfs

# Préparer les données
clean_data = prepare_price_data(data)

# Afficher un aperçu des données nettoyées
for ticker, df in clean_data.items():
    print(f"\nAperçu pour {ticker}:")
    print(df.head())
    print(f"Nombre d'observations: {len(df)}")

## 4. Stockage des Données dans SQLite
Nous utilisons SQLAlchemy pour gérer notre base de données SQLite. Les données sont structurées en plusieurs tables :
- ETFs : informations sur les ETFs
- PriceData : données historiques de prix
- PerformanceMetrics : métriques calculées

In [ ]:
# Exemple de requête pour vérifier les données stockées
query = """
SELECT p.ticker, COUNT(*) as nb_points,
       MIN(date) as debut, MAX(date) as fin
FROM price_data p
GROUP BY p.ticker
"""

# Exécuter la requête
with repository.engine.connect() as conn:
    result = pd.read_sql(query, conn)

print("\nRésumé des données stockées dans SQLite:")
print(result)

## 5. Jointure entre Deux Jeux de Données
Nous allons joindre les données de prix avec les métadonnées des ETFs pour une analyse plus complète.

In [ ]:
# Requête avec jointure pour obtenir les prix et les métadonnées
query_jointure = """
SELECT e.ticker, e.name, e.issuer, e.ter,
       p.date, p.adj_close, p.volume
FROM etfs e
JOIN price_data p ON e.ticker = p.ticker
WHERE p.date >= DATE('now', '-30 days')
ORDER BY e.ticker, p.date
"""

# Exécuter la requête
with repository.engine.connect() as conn:
    donnees_jointes = pd.read_sql(query_jointure, conn)

print("\nAperçu des données jointes:")
print(donnees_jointes.head())
print(f"\nNombre total d'observations: {len(donnees_jointes)}")

## 6. Visualisation des Données
Utilisons Plotly pour créer des visualisations interactives des données d'ETFs.

In [ ]:
# Créer un graphique des prix normalisés
def plot_normalized_prices(clean_data):
    # Normaliser les prix
    normalized_data = {}
    for ticker, df in clean_data.items():
        normalized_data[ticker] = df['adj_close'] / df['adj_close'].iloc[0] * 100
    
    # Créer le graphique
    fig = go.Figure()
    for ticker, prices in normalized_data.items():
        fig.add_trace(go.Scatter(
            x=prices.index,
            y=prices.values,
            name=ticker,
            mode='lines'
        ))
    
    fig.update_layout(
        title='Performance Relative des ETFs (Base 100)',
        xaxis_title='Date',
        yaxis_title='Prix Normalisé',
        template='plotly_white'
    )
    
    return fig

# Afficher le graphique
fig = plot_normalized_prices(clean_data)
fig.show()

## 7. Interface Utilisateur avec Streamlit
Notre application Streamlit permet aux utilisateurs d'interagir avec les données et les analyses. Voici un aperçu du code de l'interface utilisateur.

In [ ]:
# Exemple de code pour l'interface Streamlit
'''
import streamlit as st

def main():
    st.title("Analyseur d'ETFs Thématiques")
    
    # Sélection des ETFs
    etfs_selection = st.multiselect(
        "Sélectionnez les ETFs à analyser:",
        options=etfs,
        default=etfs[:2]
    )
    
    # Période d'analyse
    periode = st.selectbox(
        "Période d'analyse:",
        options=['1m', '3m', '6m', '1y', '2y', '5y'],
        index=3
    )
    
    if st.button("Analyser"):
        # Charger et afficher les données
        data = loader.fetch_multiple_etfs_data(
            tickers=etfs_selection,
            period=periode
        )
        
        # Afficher les visualisations
        clean_data = prepare_price_data(data)
        fig = plot_normalized_prices(clean_data)
        st.plotly_chart(fig)

if __name__ == "__main__":
    main()
'''

## Conclusion

Ce notebook a présenté les principales fonctionnalités de notre projet d'analyse d'ETFs :

1. Architecture modulaire avec séparation des responsabilités
2. ETL robuste pour la collecte et le traitement des données
3. Stockage efficace dans SQLite
4. Visualisations interactives avec Plotly
5. Interface utilisateur conviviale avec Streamlit

Le projet respecte les bonnes pratiques de programmation Python et utilise les outils modernes de data science.